In [28]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons

In [38]:
moons_data, moons_target = make_moons(n_samples=10000, noise=0.15, random_state=42)
m, n = moons_data.shape

In [39]:
print (moons_data.shape, moons_target.shape)

(10000, 2) (10000,)


In [40]:
scaler = StandardScaler()
moons_data_plus_bias = np.c_[np.ones((m,1)), moons_data]
scaled_moons_data_plus_bias = np.float32(scaler.fit_transform(moons_data_plus_bias))
moons_target_reshape = np.float32(moons_target.reshape(-1, 1))
print (scaled_moons_data_plus_bias.dtype)
#print (moons_target_reshape)
#print (scaled_moons_data_plus_bias)

float32


In [41]:
# control main hyper parameters
batch_size = 1000
n_batches = int(np.ceil(m/batch_size))
remainings = int(m%batch_size)
learning_rate = 0.01
n_epochs = 1000

In [42]:
def logistic_regression(theta, input_data):
    t = tf.matmul(input_data, theta)
    prob = 1.0 / (1.0 + tf.exp(-t))
    #index_true = np.where((prob >= 0.5) == True)
    #print (input_data.shape[0]) # This statement causes error! Think about it why?
    #prediction = np.zeros(input_data.shape[0])
    #for i in index_true:
    #    prediction[i] = 1
    return prob# form of array

여기에 왜 에러 생겼나 생각해보기

In [43]:
def fetch_batch(epoch, batch_index, batch_size, original_data_X, original_data_Y):
    if batch_index == n_batches - 1:
        initial = batch_index * batch_size ; final = initial + remainings
    else:
        initial = batch_index * batch_size ; final = initial + batch_size        
    X_batch = original_data_X[initial:final,:]
    Y_batch = original_data_Y[initial:final,:]
    return X_batch, Y_batch

In [63]:
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs_HW9"
logdir = "{}/run-{}/".format(root_logdir, now)
tf.reset_default_graph()

In [64]:
original_data_X = tf.constant(scaled_moons_data_plus_bias, dtype = tf.float32,  name="orig_X")
original_data_Y = tf.constant(moons_target_reshape, dtype = tf.float32,  name="orig_Y")
X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
Y = tf.placeholder(tf.float32, shape=(None, 1), name="Y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name="theta")
with tf.name_scope("loss") as scope:
    error_one = -1.0* (Y * tf.log(logistic_regression(theta, X)) + (1.0-Y)*tf.log(1.0 - logistic_regression(theta, X)))
    cost = tf.reduce_mean(error_one, name="cost")
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(cost)
init = tf.global_variables_initializer()
saver = tf.train.Saver()

cost_summary = tf.summary.scalar('cost', cost)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [65]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            #print ("Epoch", epoch, "cost = ", cost.eval()) ## This statement causes error! Think about it why?
            print ("Epoch", epoch, "cost = ", cost.eval(feed_dict={X: scaled_moons_data_plus_bias, Y: moons_target_reshape}))
            save_path = saver.save(sess, "/Users/woohyunhan/Study_ML/HW9_temp.ckpt")
        for batch_index in range(n_batches):
            X_batch, Y_batch = fetch_batch(epoch, batch_index, batch_size, scaled_moons_data_plus_bias, moons_target_reshape)
            if batch_index % 10 == 0:
                #print ("Epoch", epoch, "cost = ", cost.eval(feed_dict={X: X_batch, Y: Y_batch}))
                step = epoch * n_batches + batch_index
                summary_str = cost_summary.eval(feed_dict={X: X_batch, Y: Y_batch})
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X:X_batch, Y:Y_batch})
    best_theta = theta.eval()
    save_path = saver.save(sess, "/Users/woohyunhan/Study_ML/HW9_final.ckpt")
    file_writer.close()
    
    #print (best_theta)
    


Epoch 0 cost =  1.3846542
Epoch 100 cost =  0.26408345
Epoch 200 cost =  0.2639385
Epoch 300 cost =  0.26393732
Epoch 400 cost =  0.26393738
Epoch 500 cost =  0.2639373
Epoch 600 cost =  0.2639373
Epoch 700 cost =  0.2639373
Epoch 800 cost =  0.2639373
Epoch 900 cost =  0.2639373


Epoch마다 cost.eval을 저렇게 설정하면 안되는 이유 생각하기

In [66]:
t = np.matmul(scaled_moons_data_plus_bias, best_theta)
final_prob = 1.0 / (1.0 + np.exp(-t))
final_prob = final_prob.reshape(-1)
#final_prob = logistic_regression(best_theta, scaled_moons_data_plus_bias)
index_true = np.where((final_prob >= 0.5) == True)
#print (index_true)
prediction = np.zeros(scaled_moons_data_plus_bias.shape[0])
for i in index_true:
    prediction[i] = 1
error = np.average((np.abs((prediction - moons_target_reshape.reshape(-1)))))
print ("Accuracy = ", 1-error)
print (best_theta)

Accuracy =  0.8799
[[-0.82833076]
 [ 1.0930885 ]
 [-2.8191605 ]]
